## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [36]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [37]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet', engine='pyarrow')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet', engine='pyarrow')

num_columns = df_jan.shape[1]
print(f'The data for January have {num_columns} columns.')


The data for January have 19 columns.


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [38]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [39]:
df_jan['duration'] = (df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime).dt.total_seconds() / 60

In [40]:

# Calculate the std_dev of the column 'duration'
std_dev_duration = df_jan['duration'].std()

print(f'The standard deviation of the duration is: {std_dev_duration}')


The standard deviation of the duration is: 42.59435124195458


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [41]:

total_records = len(df_jan)
print(f'The total number of records in January is: {total_records}')

# Filter out records with duration less than 1 minute or greater than 60 minutes
df_jan_filtered = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
filtered_records = len(df_jan_filtered)
print(f'The number of records after filtering is: {filtered_records}')

# Calculate the fraction of records remaining after filtering
fraction_remaining = filtered_records / total_records
print(f'The fraction of records remaining after filtering is: {fraction_remaining:.2%}')

The total number of records in January is: 3066766
The number of records after filtering is: 3009173
The fraction of records remaining after filtering is: 98.12%


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [42]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_jan_filtered = df_jan_filtered[categorical + ['duration']]
df_jan_filtered[categorical] = df_jan_filtered[categorical].astype(str)

train_dicts = df_jan_filtered[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

y_train = df_jan_filtered['duration'].values

print(f'The shape of the feature matrix is: {X_train.shape}')

num_columns = X_train.shape[1]
print(f'This matrix has {num_columns} columns (dimensionality).')

The shape of the feature matrix is: (3009173, 515)
This matrix has 515 columns (dimensionality).


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [43]:
target = 'duration'
y_train = df_jan_filtered[target].values

# Train a linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = root_mean_squared_error(y_train, y_pred)
print(f'The RMSE of the linear regression model is: {rmse:.2f}')


The RMSE of the linear regression model is: 7.65


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [45]:
df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]
df_feb[categorical] = df_feb[categorical].astype(str)

val_dicts = df_feb[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_val = df_feb[target].values
y_pred = lr.predict(X_val)

rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))
print(f'The RMSE of the linear regression model on the validation set is: {rmse_val:.2f}')


C:\Users\Sandra\AppData\Local\Temp\ipykernel_12464\3368130309.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb[categorical] = df_feb[categorical].astype(str)


The RMSE of the linear regression model on the validation set is: 7.81
